In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
def change_brightness(img, adjustment):
    adjusted_img = img.astype(np.int32) + adjustment
    adjusted_img = np.clip(adjusted_img, 0, 255)
    return adjusted_img.astype(np.uint8)

In [ ]:
def change_contrast(img, adjustment):
    float_img = img.astype(np.float32) / 255.0
    adjusted_channels = np.clip((float_img - 0.5) * adjustment + 0.5, 0, 1)
    adjusted_img = (adjusted_channels * 255).astype(np.uint8)
    return adjusted_img

In [ ]:
def flip(img, axis='horizontal'):
    if axis == 'horizontal':
        return img[:, ::-1, :]
    else:
        return img[::-1, :, :]

In [ ]:
def convert_scale(img, method='sepia'):
    if method == 'grayscale':
        adjusted_img = np.dot(img[...,:3], [0.2989, 0.5870, 0.1140])
    else:
        sepia_mtx = np.array([[0.393, 0.769, 0.189],
                              [0.349, 0.686, 0.168],
                              [0.272, 0.534, 0.131]])
        adjusted_img = np.clip(np.dot(img, sepia_mtx.T), 0, 255)
    
    adjusted_img = np.round(adjusted_img).astype(np.uint8)
    return adjusted_img

In [ ]:
def convolution(img, kernel):
    height, width, _ = img.shape
    kernel_height, kernel_width = kernel.shape
    
    adjusted_img = np.zeros((height, width, 3), dtype=float) 
    
    for i in range(kernel_height//2, height-kernel_height//2-1):
        for j in range(kernel_width//2, width-kernel_width//2-1):
            window = img[i-kernel_height//2 : i+kernel_height//2+1,j-kernel_width//2 : j+kernel_width//2+1]
            adjusted_img[i, j] = [int((window[:,:,k] * kernel).sum()) for k in range(3)]
      
    adjusted_img = np.clip(adjusted_img, 0, 255)
    return adjusted_img.astype(np.uint8)

In [ ]:
def center_crop(img, crop_shape):
    height, width, _ = img.shape
    crop_height, crop_width = crop_shape

    start_height = (height - crop_height) // 2
    start_width = (width - crop_width) // 2

    end_height = start_height + crop_height
    end_width = start_width + crop_width

    cropped_img = img[start_height:end_height, start_width:end_width]
    return cropped_img

In [ ]:
def circle_frame_crop(img, center, radius):
    yy, xx = np.ogrid[:img.shape[0], :img.shape[1]]
    cy, cx = center
    
    circle_mask = (yy - cy)**2 + (xx - cx)**2 <= radius**2

    cropped_img = np.copy(img)
    cropped_img[~circle_mask] = 0

    return cropped_img

In [ ]:
def ellipse_frame_crop(img, center, major_axis, minor_axis, deg_angle):
    yy, xx = np.ogrid[:img.shape[0], :img.shape[1]]
    cy, cx = center
    rad_angle = np.deg2rad(deg_angle)

    ellipse_mask = ((xx - cx) * np.cos(rad_angle) + (yy - cy) * np.sin(rad_angle))**2 / (major_axis**2) + ((xx - cx) * np.sin(rad_angle) - (yy - cy) * np.cos(rad_angle))**2 / (minor_axis**2) <= 1

    cropped_img = np.copy(img)
    cropped_img[~ellipse_mask] = 0

    return cropped_img

In [ ]:
def x_ellipse_frame_crop(img, center, major_axis, minor_axis):
    yy, xx = np.ogrid[:img.shape[0], :img.shape[1]]
    cy, cx = center
    
    deg_angle = 45.0
    rad_angle = np.deg2rad(deg_angle)
    ellipse_mask_1 = ((xx - cx) * np.cos(rad_angle) + (yy - cy) * np.sin(rad_angle))**2 / (major_axis**2) + ((xx - cx) * np.sin(rad_angle) - (yy - cy) * np.cos(rad_angle))**2 / (minor_axis**2) <= 1

    deg_angle = 135.0
    rad_angle = np.deg2rad(deg_angle)
    ellipse_mask_2 = ((xx - cx) * np.cos(rad_angle) + (yy - cy) * np.sin(rad_angle))**2 / (major_axis**2) + ((xx - cx) * np.sin(rad_angle) - (yy - cy) * np.cos(rad_angle))**2 / (minor_axis**2) <= 1
    
    cropped_img = np.copy(img)
    cropped_img[~(ellipse_mask_1 | ellipse_mask_2)] = 0
    
    return cropped_img

In [ ]:
def menu_choice():
    print('1. Change brightness')
    print('2. Change contrast')
    print('3. Flip image')
    print('4. Convert to grayscale/sepia')
    print('5. Blur/sharpen')
    print('6. Center crop')
    print('7. Circle frame crop')
    print('8. Double elip frame crop')
    print('0. Change all')
    choice = int(input('Your choice: '))
    
    if choice == 3:
        print('\n1. Flip vertically')
        print('2. Flip horizontally')
        sub_choice = int(input('Your choice: '))
        choice = 9 if sub_choice == 1 else 10
    elif choice == 4:
        print('\n1. Grayscale')
        print('2. Sepia')
        sub_choice = int(input('Your choice: '))
        choice = 11 if sub_choice == 1 else 12
    elif choice == 5:
        print('\n1. Blur')
        print('2. Sharpen')
        sub_choice = int(input('Your choice: '))
        choice = 13 if sub_choice == 1 else 14
    return choice

In [ ]:
# kernel for blurring image
GAUSSIAN_KERNEL_3 = 1/16 * np.array([[1,2,1],
                                     [2,4,2],
                                     [1,2,1]])
GAUSSIAN_KERNEL_5 = 1/256 * np.array([[1, 4, 6, 4, 1],
                                      [4, 16, 24, 16, 4],
                                      [6, 24, 36, 24, 6],
                                      [4, 16, 24, 16, 4],
                                      [1, 4, 6, 4, 1]])
# kernel for sharpening image
SHARPEN_KERNEL_3 = np.array([[0,-1,0],
                             [-1,5,-1],
                             [0,-1,0]])
SHARPEN_KERNEL_5 = -1/256 * np.array([[1, 4, 6, 4, 1],
                                      [4, 16, 24, 16, 4],
                                      [6, 24, -476, 24, 6],
                                      [4, 16, 24, 16, 4],
                                      [1, 4, 6, 4, 1]])

In [ ]:
def main():
    img_path = str(input('Path to image: '))
    try:
        image = Image.open(img_path)
    except:
        print('can not open image!')
        return
    
    org_img = np.array(image)
    
    choice = menu_choice()
    processed_images = []
    processed_mechanism = []
    
    if choice == 1 or choice == 0:
        adjustment = int(input('Input brightness change: '))
        prd_img = change_brightness(org_img.copy(), adjustment)
        plt.imsave('output_brightness_change.png', prd_img, cmap='gray')
        processed_images.append(prd_img)
        processed_mechanism.append('Brightness changed')
        
    if choice == 2 or choice == 0:
        adjustment = input('Input contrast change: ')
        prd_img = change_contrast(org_img, float(adjustment))
        plt.imsave('output_contrast_change.png', prd_img, cmap='gray')
        processed_images.append(prd_img)
        processed_mechanism.append('Contrast changed')
    
    if choice == 9 or choice == 0:
        prd_img = flip(org_img, 'vertical')
        plt.imsave('output_vertical_flip.png', prd_img, cmap='gray')
        processed_images.append(prd_img)
        processed_mechanism.append('Vertical flip')
    
    if choice == 10 or choice == 0:
        prd_img = flip(org_img, 'horizontal')
        plt.imsave('output_horizontal_flip.png', prd_img, cmap='gray')
        processed_images.append(prd_img)
        processed_mechanism.append('Horizontal flip')
    
    if choice == 11 or choice == 0:
        prd_img = convert_scale(org_img, 'grayscale')
        plt.imsave('output_grayscale.png', prd_img, cmap='gray')
        processed_images.append(prd_img)
        processed_mechanism.append('Grayscaled converted')
    
    if choice == 12 or choice == 0:
        prd_img = convert_scale(org_img, 'sepia')
        plt.imsave('output_sepia.png', prd_img, cmap='gray')
        processed_images.append(prd_img)
        processed_mechanism.append('Sepia converted')
    
    if choice == 13 or choice == 0:
        size = int(input('Kernel size for blurring, 3 or 5: '))
        prd_img = convolution(org_img, GAUSSIAN_KERNEL_3 if size == 3 else GAUSSIAN_KERNEL_5)
        plt.imsave('output_blur.png', prd_img, cmap='gray')
        processed_images.append(prd_img)
        processed_mechanism.append('Blurred')
    
    if choice == 14 or choice == 0:
        size = int(input('Kernel size for sharpenning, 3 or 5: '))
        prd_img = convolution(org_img, SHARPEN_KERNEL_3 if size == 3 else SHARPEN_KERNEL_5)
        plt.imsave('output_sharpen.png', prd_img, cmap='gray')
        processed_images.append(prd_img)
        processed_mechanism.append('Sharpenned')
    
    if choice == 6 or choice == 0:
        height = int(input('Crop height: '))
        width = int(input('Crop width: '))
        prd_img = center_crop(org_img, (height, width))
        plt.imsave('output_center_crop.png', prd_img, cmap='gray')
        processed_images.append(prd_img)
        processed_mechanism.append('Center cropped')
    
    if choice == 7 or choice == 0:
        cx = int(input('Center coordinate\'s x: '))
        cy = int(input('Center coordinate\'s y: '))
        center = (cx, cy)
        radius = int(input('Circle frame radius: '))
        prd_img = circle_frame_crop(org_img, center, radius)
        plt.imsave('output_circle_crop.png', prd_img, cmap='gray')
        processed_images.append(prd_img)
        processed_mechanism.append('Circle cropped')
    
    if choice == 8 or choice == 0:
        cx = int(input('Center coordinate\'s x: '))
        cy = int(input('Center coordinate\'s y: '))
        center = (cx, cy)
        major_axis = int(input('Major axis length: '))
        minor_axis = int(input('Minor axis length: '))
        prd_img = x_ellipse_frame_crop(org_img, center, major_axis, minor_axis)
        plt.imsave('output_x_ellipse_crop.png', prd_img, cmap='gray')
        processed_images.append(prd_img)
        processed_mechanism.append('Crossover ellipse cropped')
    
    num_images = len(processed_images)
    fig, axs = plt.subplots(num_images + 1, 1, figsize=(400, 100) if num_images != 1 else (26,14))
    axs[0].imshow(org_img, cmap='gray')
    axs[0].set_title("Original Image")
    axs[0].axis("off")

    for i in range(num_images):
        axs[i + 1].imshow(processed_images[i], cmap='gray')
        axs[i + 1].set_title(processed_mechanism[i])
        axs[i + 1].axis("off")

    plt.tight_layout()
    plt.show()
    
if __name__ == "__main__":
    main()